<a href="https://colab.research.google.com/github/ErdemAslans/Document-Processing-Pipeline-with-Swin-Transformer-OCR-and-LLM-Analysis/blob/main/Document_Processing_Pipeline_with_Swin_Transformer%2C_OCR%2C_and_LLM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unstructured gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 53.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of unstructured-client to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of unstructured-client to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.6 MB/s eta 0:00:

In [2]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 24.0 MB/s eta 0:00:00


In [3]:
!pip install 'unstructured[image]'
!apt-get install -y tesseract-ocr
!pip install pytesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.1/526.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
^C
^C


In [1]:
import os
import torch
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import SwinForImageClassification, AutoProcessor, pipeline
from unstructured.partition.auto import partition
from unstructured.staging.base import elements_to_text
import requests
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import gradio as gr
import tempfile
import time
import json
import base64
from io import BytesIO
import shutil
import random
from tqdm.auto import tqdm

def mount_drive():
    drive.mount('/content/drive')
    print("Google Drive bağlandı.")

CLASSES = [
    'letter', 'form', 'email', 'handwritten', 'advertisement',
    'scientific_report', 'scientific_publication', 'specification',
    'file_folder', 'news_article', 'budget', 'invoice',
    'presentation', 'questionnaire', 'resume', 'memo'
]


In [2]:
def prepare_dataset(data_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=42):
    """
    Veri setini train/val/test alt klasörlerine böler.

    Args:
        data_dir: RVL_CDIP_small veri setinin yolu
        train_ratio: Eğitim için kullanılacak veri oranı
        val_ratio: Doğrulama için kullanılacak veri oranı
        test_ratio: Test için kullanılacak veri oranı
        seed: Rastgelelik için tohum değeri
    """
    random.seed(seed)

    train_dir = os.path.join(data_dir, 'train')
    val_dir = os.path.join(data_dir, 'val')
    test_dir = os.path.join(data_dir, 'test')

    for directory in [train_dir, val_dir, test_dir]:
        if os.path.exists(directory):
            shutil.rmtree(directory)
        os.makedirs(directory)

    for class_name in CLASSES:
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

    # Sınıf başına dosya sayılarını tutacak sayaçlar
    class_counts = {cls: {"train": 0, "val": 0, "test": 0} for cls in CLASSES}

    for class_name in tqdm(CLASSES, desc="Sınıflar hazırlanıyor"):
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):
            print(f"Uyarı: {class_name} sınıfı için klasör bulunamadı.")
            continue

        files = [f for f in os.listdir(class_dir) if f.endswith(('.tif', '.jpg', '.png', '.jpeg', '.pdf'))]

        # Sınıfta çok az dosya varsa bile veri kümesi dengeli olsun
        if len(files) < 10:
            print(f"Uyarı: {class_name} sınıfında sadece {len(files)} örnek var. Veri artırma düşünülebilir.")

        random.shuffle(files)

        n_files = len(files)
        n_train = int(n_files * train_ratio)
        n_val = int(n_files * val_ratio)

        for i, file in enumerate(files):
            src = os.path.join(class_dir, file)
            if i < n_train:
                dst = os.path.join(train_dir, class_name, file)
                class_counts[class_name]["train"] += 1
            elif i < n_train + n_val:
                dst = os.path.join(val_dir, class_name, file)
                class_counts[class_name]["val"] += 1
            else:
                dst = os.path.join(test_dir, class_name, file)
                class_counts[class_name]["test"] += 1
            shutil.copy2(src, dst)

    # Özet tablo oluştur
    summary_table = []
    for cls in CLASSES:
        summary_table.append([
            cls,
            class_counts[cls]["train"],
            class_counts[cls]["val"],
            class_counts[cls]["test"],
            sum(class_counts[cls].values())
        ])

    summary_df = pd.DataFrame(
        summary_table,
        columns=["Sınıf", "Eğitim", "Doğrulama", "Test", "Toplam"]
    )

    print("\nVeri Seti Dağılımı:")
    print(summary_df)
    print("\nToplam:")
    print(f"Eğitim: {summary_df['Eğitim'].sum()} örnek")
    print(f"Doğrulama: {summary_df['Doğrulama'].sum()} örnek")
    print(f"Test: {summary_df['Test'].sum()} örnek")
    print(f"Genel Toplam: {summary_df['Toplam'].sum()} örnek")

    print("\nVeri seti hazırlama tamamlandı.")
    print(f"Eğitim: {train_dir}")
    print(f"Doğrulama: {val_dir}")
    print(f"Test: {test_dir}")

    return train_dir, val_dir, test_dir


In [3]:
class DocumentDataset(Dataset):
    def __init__(self, root_dir, processor, class_to_idx=None, transform=None, is_training=False):
        self.root_dir = root_dir
        self.processor = processor
        self.transform = transform
        self.samples = []
        self.is_training = is_training

        # İstatistikler için sayaçlar
        self.tif_count = 0
        self.other_count = 0
        self.error_count = 0

        if class_to_idx is None:
            self.class_to_idx = {cls_name: i for i, cls_name in enumerate(CLASSES)}
        else:
            self.class_to_idx = class_to_idx

        # Sınıfları ve örnekleri yükle
        valid_classes = []
        for class_name in os.listdir(root_dir):
            class_dir = os.path.join(root_dir, class_name)
            if os.path.isdir(class_dir) and class_name in self.class_to_idx:
                valid_classes.append(class_name)
                for filename in os.listdir(class_dir):
                    # Büyük/küçük harf duyarsız kontrol
                    lower_filename = filename.lower()
                    if lower_filename.endswith(('.tif', '.tiff')):
                        self.tif_count += 1
                        image_path = os.path.join(class_dir, filename)
                        self.samples.append((image_path, self.class_to_idx.get(class_name, 0)))
                    elif lower_filename.endswith(('.jpg', '.jpeg', '.png', '.pdf')):
                        self.other_count += 1
                        image_path = os.path.join(class_dir, filename)
                        self.samples.append((image_path, self.class_to_idx.get(class_name, 0)))

        print(f"Veri seti yüklendi: {len(self.samples)} belge, {len(valid_classes)} sınıf")
        print(f"TIF dosyaları: {self.tif_count}, Diğer dosyalar: {self.other_count}")

        if len(valid_classes) < len(CLASSES):
            missing = set(CLASSES) - set(valid_classes)
            print(f"Uyarı: {missing} sınıfları bulunamadı.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, label = self.samples[idx]

        try:
            # Görüntüyü yükle
            image = self._load_image(image_path)

            # ÖNEMLİ DÜZELTME: Ya transform kullan ya da processor, ikisini birlikte değil
            if self.is_training and self.transform:
                # Eğitim sırasında veri artırma için transform kullan
                pixel_values = self.transform(image)
            else:
                # Normal işleme için processor kullan
                # Processor içinde zaten dönüşüm var, o yüzden processor'ı direkt PIL görüntüsüne uygula
                inputs = self.processor(images=image, return_tensors="pt")
                pixel_values = inputs["pixel_values"].squeeze()

            return {
                'pixel_values': pixel_values,
                'labels': torch.tensor(label, dtype=torch.long),
                'path': image_path
            }
        except Exception as e:
            self.error_count += 1
            if self.error_count <= 5:  # Sadece ilk birkaç hatayı göster
                print(f"Görüntü işleme hatası ({image_path}): {e}")

            # Hata durumunda varsayılan tensör döndür
            return {
                'pixel_values': torch.zeros((3, 224, 224)),
                'labels': torch.tensor(label, dtype=torch.long),
                'path': image_path
            }

    def _load_image(self, image_path):
        """TIF formatını destekleyen görüntü yükleme fonksiyonu"""
        try:
            # PIL ile görüntüyü açma
            with Image.open(image_path) as img:
                return img.convert('RGB')
        except Exception as e:
            print(f"PIL ile yükleme hatası ({image_path}): {e}")

            # TIF dosyaları için alternatif yöntem
            try:
                import imageio.v2 as imageio  # v2 kullan (deprecation uyarısını önlemek için)
                img_array = imageio.imread(image_path)

                # 2D (gri tonlamalı) bir görüntüyse 3 kanala dönüştür
                if len(img_array.shape) == 2:
                    img_array = np.stack([img_array, img_array, img_array], axis=2)
                elif len(img_array.shape) == 3 and img_array.shape[2] > 3:
                    # RGBA veya başka çok kanallı formatta ise, ilk 3 kanalı al
                    img_array = img_array[:, :, :3]

                # NumPy dizisini PIL görüntüsüne dönüştür
                return Image.fromarray(np.uint8(img_array))
            except Exception as e2:
                print(f"Alternatif yükleme hatası ({image_path}): {e2}")
                # En kötü durumda gri bir görüntü döndür
                return Image.new('RGB', (224, 224), color='gray')

In [ ]:
class SwinImageProcessor:
    def __init__(self, image_size=224, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
        self.image_size = image_size
        self.mean = mean
        self.std = std

        # Temel dönüşüm
        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])

        # Eğitim için veri artırma dönüşümleri
        self.train_transform = transforms.Compose([
            transforms.Resize((int(image_size*1.1), int(image_size*1.1))),
            transforms.RandomCrop(image_size),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])

    def __call__(self, images, return_tensors="pt"):
        """
        Görüntüleri işleyip model için hazırlar - TIF formatları için optimize edilmiş
        """
        if not isinstance(images, list):
            images = [images]

        pixel_values = []
        for image in images:
            try:
                # PIL.Image kontrolünü kaldırıldı, herhangi bir formattaki görüntüyü işlemeye çalışacak
                pixel_value = self.transform(image)
                pixel_values.append(pixel_value)
            except Exception as e:
                print(f"Görüntü dönüşüm hatası: {e}")
                # Hata durumunda varsayılan bir tensör oluştur
                pixel_values.append(torch.zeros((3, self.image_size, self.image_size)))

        # Tensörleri yığınla
        if return_tensors == "pt":
            pixel_values = torch.stack(pixel_values)

        return {"pixel_values": pixel_values}

    def get_train_transform(self):
        """Eğitim için veri artırma dönüşümlerini döndürür"""
        return self.train_transform

In [ ]:
class DocumentClassifier:
    def __init__(self, num_classes=16, pretrained_model="microsoft/swin-base-patch4-window7-224-in22k", model_path='/content/drive/MyDrive/RVL_CDIP_small/best_document_classifier.pth'):
        from transformers import SwinForImageClassification
        self.processor = SwinImageProcessor(image_size=224)

        self.device = torch.device("cpu")

        # Önce kontrol et: Eğitilmiş model var mı?
        if model_path and os.path.exists(model_path):
            print(f"Eğitilmiş model bulundu, yükleniyor: {model_path}")

            # Doğrudan pretrained modeli çağırıp sonra state_dict değiştirmek yerine
            # Modelin yapısına göre state_dict'i yükle

            # Önce model state_dict'ini yükle
            checkpoint = torch.load(model_path, map_location=self.device)

            # Modeli oluştur - API değişikliklerinden etkilenmemek için from_pretrained kullan
            if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
                # Checkpoint bir dictionary ise ve içinde model_state_dict varsa

                # Geçici bir model oluştur ve yapısını kopyala
                print("Geçici model yapısı oluşturuluyor...")
                temp_model = SwinForImageClassification.from_pretrained(
                    pretrained_model,
                    num_labels=num_classes,
                    ignore_mismatched_sizes=True
                )

                # Yeni bir model oluştur ve sadece yapılandırmasını kullan
                self.model = SwinForImageClassification(temp_model.config)

                # Eğitilmiş model ağırlıklarını yükle
                self.model.load_state_dict(checkpoint['model_state_dict'])
                print("Model ağırlıkları başarıyla yüklendi!")

                # Geçici modeli bellekten temizle
                import gc
                del temp_model
                gc.collect()
                torch.cuda.empty_cache()
            else:

                print("Geçici model yapısı oluşturuluyor...")
                temp_model = SwinForImageClassification.from_pretrained(
                    pretrained_model,
                    num_labels=num_classes,
                    ignore_mismatched_sizes=True
                )

                # Yeni model oluştur
                self.model = SwinForImageClassification(temp_model.config)

                # Ağırlıkları yükle
                self.model.load_state_dict(checkpoint)
                print("Model ağırlıkları başarıyla yüklendi!")

                # Geçici modeli temizle
                import gc
                del temp_model
                gc.collect()
                torch.cuda.empty_cache()
        else:
            # Eğitilmiş model yoksa pretrained modeli indir
            print(f"Eğitilmiş model bulunamadı. Pretrained model indiriliyor.")
            self.model = SwinForImageClassification.from_pretrained(
                pretrained_model,
                num_labels=num_classes,
                ignore_mismatched_sizes=True
            )

        print(f"Model {self.device} üzerinde çalışacak")
        self.model.to(self.device)
        self.idx_to_class = {i: cls for i, cls in enumerate(CLASSES)}

    def train(self, train_loader, val_loader, num_epochs=10, learning_rate=2e-5, weight_decay=0.01, progress=None):
        """
        İyileştirilmiş eğitim fonksiyonu:
        - Weight decay eklenmiş AdamW optimizer
        - Cosine annealing öğrenme oranı çizelgesi
        - Karışıklık matrisi izleme
        - Erken durdurma
        - Daha detaylı metrikler
        """
        optimizer = torch.optim.AdamW(
            self.model.parameters(),
            lr=learning_rate,
            weight_decay=weight_decay
        )

        # Öğrenme oranı çizelgesi
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=num_epochs
        )

        best_accuracy = 0.0
        best_f1 = 0.0
        patience = 3
        patience_counter = 0

        train_losses = []
        val_accuracies = []
        val_f1_scores = []

        total_steps = num_epochs * len(train_loader)
        current_step = 0

        # Eğitim başlangıç zamanı
        start_time = time.time()

        for epoch in range(num_epochs):
            epoch_start = time.time()

            # TRAINING PHASE
            self.model.train()
            running_loss = 0.0
            train_correct = 0
            train_total = 0

            progress_bar = tqdm(train_loader, desc=f"Eğitim Epoch {epoch+1}/{num_epochs}")
            for batch in progress_bar:
                pixel_values = batch['pixel_values'].to(self.device)
                labels = batch['labels'].to(self.device)

                optimizer.zero_grad()

                outputs = self.model(pixel_values=pixel_values, labels=labels)
                loss = outputs.loss

                loss.backward()
                optimizer.step()

                running_loss += loss.item()

                # Eğitim doğruluğunu takip et
                _, predicted = torch.max(outputs.logits, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()

                current_step += 1
                progress_bar.set_postfix({'loss': loss.item()})
                if progress is not None:
                    progress((current_step / total_steps) * 100, f"Epoch {epoch+1}/{num_epochs}, Batch {current_step % len(train_loader)}/{len(train_loader)}")

            epoch_loss = running_loss / len(train_loader)
            train_losses.append(epoch_loss)
            train_accuracy = 100 * train_correct / train_total

            # VALIDATION PHASE
            self.model.eval()
            all_preds = []
            all_labels = []
            val_loss = 0.0

            with torch.no_grad():
                for batch in tqdm(val_loader, desc=f"Doğrulama Epoch {epoch+1}/{num_epochs}"):
                    pixel_values = batch['pixel_values'].to(self.device)
                    labels = batch['labels'].to(self.device)

                    outputs = self.model(pixel_values=pixel_values, labels=labels)
                    val_loss += outputs.loss.item()

                    _, predicted = torch.max(outputs.logits, 1)

                    all_preds.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

            # Doğrulama metrikleri
            val_epoch_loss = val_loss / len(val_loader)
            accuracy = 100 * np.mean(np.array(all_preds) == np.array(all_labels))
            val_accuracies.append(accuracy)

            # F1 skoru hesapla
            from sklearn.metrics import f1_score
            f1 = f1_score(all_labels, all_preds, average='weighted')
            val_f1_scores.append(f1)

            # Doğrulama karışıklık matrisini hesapla
            cm = confusion_matrix(all_labels, all_preds, labels=range(len(CLASSES)))

            # Epoch sonuçlarını yazdır
            epoch_time = time.time() - epoch_start
            log_msg = (f'Epoch [{epoch+1}/{num_epochs}], '
                      f'Train Loss: {epoch_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
                      f'Val Loss: {val_epoch_loss:.4f}, Val Acc: {accuracy:.2f}%, '
                      f'F1: {f1:.4f}, Time: {epoch_time:.1f}s')
            print(log_msg)

            # En iyi modeli kaydet
            improvement = False
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                improvement = True
                print(f"✓ Doğruluk iyileşti: {best_accuracy:.2f}%")

            if f1 > best_f1:
                best_f1 = f1
                improvement = True
                print(f"✓ F1 skoru iyileşti: {best_f1:.4f}")

            if improvement:
                model_path = os.path.join(os.getcwd(), '/content/drive/MyDrive/RVL_CDIP_small/best_document_classifier.pth')
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'train_loss': epoch_loss,
                    'val_loss': val_epoch_loss,
                    'accuracy': accuracy,
                    'f1': f1
                }, model_path)
                print(f"✓ En iyi model kaydedildi: {model_path}")
                patience_counter = 0
            else:
                patience_counter += 1
                print(f"! İyileşme yok. Kalan sabır: {patience - patience_counter}")

            # Erken durdurma kontrolü
            if patience_counter >= patience:
                print(f"! {patience} epoch boyunca iyileşme olmadı. Eğitim erken durduruluyor.")
                break

            scheduler.step()

            # Belleği temizle
            torch.cuda.empty_cache()

        # Eğitim süresi
        total_time = time.time() - start_time
        print(f"Toplam eğitim süresi: {total_time/60:.2f} dakika")

        # Eğitim sonuçlarını grafikle
        plot_path = self._plot_training_results(train_losses, val_accuracies, val_f1_scores, num_epochs)

        return {
            'train_losses': train_losses,
            'val_accuracies': val_accuracies,
            'val_f1_scores': val_f1_scores,
            'best_accuracy': best_accuracy,
            'best_f1': best_f1,
            'plot_path': plot_path,
            'total_time': total_time
        }

    def _plot_training_results(self, train_losses, val_accuracies, val_f1_scores, num_epochs):
        plt.figure(figsize=(15, 5))

        # Kayıp grafiği
        plt.subplot(1, 3, 1)
        plt.plot(range(1, len(train_losses)+1), train_losses, 'b-', label='Eğitim Kaybı')
        plt.title('Eğitim Kaybı')
        plt.xlabel('Epok')
        plt.ylabel('Kayıp')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend()

        # Doğruluk grafiği
        plt.subplot(1, 3, 2)
        plt.plot(range(1, len(val_accuracies)+1), val_accuracies, 'r-', label='Doğrulama Doğruluğu')
        plt.title('Doğrulama Doğruluğu')
        plt.xlabel('Epok')
        plt.ylabel('Doğruluk (%)')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend()

        # F1 skor grafiği
        plt.subplot(1, 3, 3)
        plt.plot(range(1, len(val_f1_scores)+1), val_f1_scores, 'g-', label='F1 Skoru')
        plt.title('Doğrulama F1 Skoru')
        plt.xlabel('Epok')
        plt.ylabel('F1 Skoru')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend()

        plt.tight_layout()

        # Geçici dosya oluştur
        plot_path = os.path.join(os.getcwd(), 'training_results.png')
        plt.savefig(plot_path)
        plt.close()

        return plot_path

    def load_best_model(self, model_path='/content/drive/MyDrive/RVL_CDIP_small/best_document_classifier.pth'):
        try:
            checkpoint = torch.load(model_path, map_location=self.device)

            # Eski model kayıt formatı kontrolü (state_dict doğrudan kaydedildiyse)
            if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
                self.model.load_state_dict(checkpoint['model_state_dict'])
                print(f"Model yüklendi: {model_path} (Epoch: {checkpoint.get('epoch', 'Bilinmiyor')})")
                print(f"Doğruluk: {checkpoint.get('accuracy', 'Bilinmiyor'):.2f}%, F1: {checkpoint.get('f1', 'Bilinmiyor'):.4f}")
            else:
                self.model.load_state_dict(checkpoint)
                print(f"Model yüklendi: {model_path} (Eski format)")

            self.model.eval()
            return True
        except Exception as e:
            print(f"Model yüklenirken hata oluştu: {e}")
            return False

    def predict(self, image_path):
        try:
            image = Image.open(image_path).convert('RGB')
            inputs = self.processor(images=image, return_tensors="pt")
            pixel_values = inputs["pixel_values"].to(self.device)

            with torch.no_grad():
                outputs = self.model(pixel_values=pixel_values)
                probs = torch.nn.functional.softmax(outputs.logits, dim=1)

                # En yüksek olasılığa sahip sınıfı ve güveni al
                confidence, predicted_class_idx = torch.max(probs, 1)
                predicted_class = self.idx_to_class[predicted_class_idx.item()]
                confidence = confidence.item()

                # Tüm sınıf olasılıklarını al
                class_probs = {self.idx_to_class[i]: prob.item() for i, prob in enumerate(probs[0])}

                # Olasılıkları sırala
                sorted_probs = sorted(class_probs.items(), key=lambda x: x[1], reverse=True)

                return {
                    'class': predicted_class,
                    'confidence': confidence,
                    'all_probs': class_probs,
                    'sorted_probs': sorted_probs
                }
        except Exception as e:
            print(f"Tahmin hatası: {e}")
            return {
                'class': 'error',
                'confidence': 0.0,
                'error': str(e),
                'all_probs': {},
                'sorted_probs': []
            }

    def evaluate(self, test_loader, progress=None):
        self.model.eval()
        all_preds = []
        all_labels = []
        all_probs = []

        with torch.no_grad():
            total_batches = len(test_loader)
            for i, batch in enumerate(tqdm(test_loader, desc="Değerlendirme")):
                pixel_values = batch['pixel_values'].to(self.device)
                labels = batch['labels'].to(self.device)

                outputs = self.model(pixel_values=pixel_values)
                logits = outputs.logits
                probs = torch.nn.functional.softmax(logits, dim=1)
                _, predicted = torch.max(logits, 1)

                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_probs.extend(probs.cpu().numpy())

                # İlerleme durumunu güncelle
                if progress is not None:
                    progress((i / total_batches) * 100, f"Değerlendiriliyor: Batch {i+1}/{total_batches}")

        # Sınıflandırma raporu oluştur
        report = classification_report(
            all_labels,
            all_preds,
            target_names=CLASSES,
            output_dict=True
        )

        # Karışıklık matrisi oluştur
        cm = confusion_matrix(all_labels, all_preds)

        # ROC eğrisi için gerekli hesaplamaları yap
        from sklearn.preprocessing import label_binarize
        from sklearn.metrics import roc_curve, auc

        y_test_bin = label_binarize(all_labels, classes=range(len(CLASSES)))

        # Grafikleri oluştur
        cm_plot_path, f1_plot_path, roc_plot_path = self._plot_evaluation_results(cm, report, all_labels, all_probs)

        # Hatalı sınıflandırılan örnekleri incele
        misclassified = np.where(np.array(all_preds) != np.array(all_labels))[0]

        return {
            'report': report,
            'confusion_matrix': cm,
            'cm_plot_path': cm_plot_path,
            'f1_plot_path': f1_plot_path,
            'roc_plot_path': roc_plot_path,
            'misclassified_indices': misclassified
        }

    def _plot_evaluation_results(self, cm, report, all_labels, all_probs):
        # 1. Karışıklık Matrisi
        plt.figure(figsize=(12, 10))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CLASSES, yticklabels=CLASSES)
        plt.title('Karmaşıklık Matrisi')
        plt.xlabel('Tahmin Edilen Etiket')
        plt.ylabel('Gerçek Etiket')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        cm_plot_path = os.path.join(os.getcwd(), 'confusion_matrix.png')
        plt.savefig(cm_plot_path)
        plt.close()

        # 2. F1 Skorları
        df_report = pd.DataFrame(report).transpose()
        plt.figure(figsize=(12, 6))
        bar_plot = sns.barplot(x=df_report.index[:-3], y=df_report['f1-score'][:-3])

        # Değerleri çubukların üzerine ekle
        for i, p in enumerate(bar_plot.patches):
            bar_plot.annotate(f"{p.get_height():.2f}",
                       (p.get_x() + p.get_width() / 2., p.get_height()),
                       ha = 'center', va = 'bottom',
                       xytext = (0, 5),
                       textcoords = 'offset points')

        plt.title('Sınıf Bazında F1 Skorları')
        plt.xlabel('Belge Sınıfı')
        plt.ylabel('F1 Skoru')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        f1_plot_path = os.path.join(os.getcwd(), 'f1_scores.png')
        plt.savefig(f1_plot_path)
        plt.close()

        # 3. ROC Eğrisi (Bire-karşı-hepsi yaklaşımı)
        from sklearn.preprocessing import label_binarize
        from sklearn.metrics import roc_curve, auc

        # Çok sınıflı ROC analizi - Her sınıfın bireysel ROC eğrisi
        plt.figure(figsize=(12, 10))

        y_bin = label_binarize(all_labels, classes=range(len(CLASSES)))
        n_classes = y_bin.shape[1]

        # Compute ROC curve and ROC area for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()

        # Her sınıf için sadece birkaç sınıfı çiz (16 sınıfın hepsi grafiği karmaşık yapabilir)
        classes_to_plot = min(8, len(CLASSES))
        for i in range(classes_to_plot):
            fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], np.array(all_probs)[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
            plt.plot(fpr[i], tpr[i], lw=2,
                     label=f'{CLASSES[i]} (AUC = {roc_auc[i]:.2f})')

        plt.plot([0, 1], [0, 1], 'k--', lw=2)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('Yanlış Pozitif Oranı')
        plt.ylabel('Doğru Pozitif Oranı')
        plt.title('Seçilmiş Sınıflar için ROC Eğrisi')
        plt.legend(loc="lower right")
        plt.tight_layout()

        roc_plot_path = os.path.join(os.getcwd(), 'roc_curves.png')
        plt.savefig(roc_plot_path)
        plt.close()

        return cm_plot_path, f1_plot_path, roc_plot_path

In [ ]:
# DeepSeek LLM ile analiz sınıfı
class DocumentAnalyzer:
    def __init__(self, model="deepseek-ai/deepseek-llm-7b-chat"):
        self.model = model
        self.timeout = 120  # LLM analizi için 120 saniyelik zaman aşımı
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Kullanılan cihaz: {device}")


        try:
            # Yerel model pipeline'ı
            self.pipe = pipeline(
                "text-generation",
                model=model,
                torch_dtype=torch.bfloat16,
                device_map="auto",
            )
            print(f"DeepSeek LLM modeli başarıyla yüklendi: {model}")
        except Exception as e:
            print(f"DeepSeek LLM modeli yüklenirken hata oluştu: {e}")
            self.pipe = None

    def analyze_document(self, text, document_class, prompt_template=None):
        """
        Belge metnini analiz et ve önemli bilgileri çıkar
        Gelişmiş hata yakalama ve zaman aşımı kontrolü içerir
        """
        import threading
        import time
        from queue import Queue

        # Sonuç ve hata kuyrukları
        result_queue = Queue()
        exception_queue = Queue()

        # Prompt şablonunu belirle
        if prompt_template is None:
            # Sınıf-spesifik özel şablonlar
            class_specific_templates = {
                'letter': """
                Bu belge "letter" (mektup) olarak sınıflandırılmıştır.
                Lütfen aşağıdaki mektubu analiz et ve şu bilgileri çıkar:

                1. Mektubun kimden geldiği ve kime hitap ettiği
                2. Mektubun yazılış tarihi
                3. Mektubun ana konusu ve amacı
                4. Mektupta belirtilen önemli bilgiler
                5. Mektupta bahsedilen kişiler, kurumlar veya yerler
                6. Mektubun tonu ve üslubu (resmi, gayri resmi, iş mektubu vb.)
                7. Mektubun içeriğinin kısa bir özeti

                Mektup İçeriği:
                {text}
                """,

                'form': """
                Bu belge "form" (form) olarak sınıflandırılmıştır.
                Lütfen aşağıdaki formu analiz et ve şu bilgileri çıkar:

                1. Formun türü ve amacı
                2. Formda bulunan ana bölümler
                3. Formdaki önemli alanlar ve bilgiler (varsa)
                4. Form ile ilgili kurumsal bilgiler
                5. Formun doldurulması için gereken bilgiler
                6. Formun genel yapısı ve organizasyonu
                7. Formun içeriğinin kısa bir özeti

                Form İçeriği:
                {text}
                """,

                'invoice': """
                Bu belge "invoice" (fatura) olarak sınıflandırılmıştır.
                Lütfen aşağıdaki faturayı analiz et ve şu bilgileri çıkar:

                1. Fatura tarihi ve numarası
                2. Satıcı/sağlayıcı bilgileri
                3. Alıcı bilgileri
                4. Fatura kalemleri (ürünler/hizmetler ve fiyatları)
                5. Toplam tutar, vergi tutarı ve varsa indirimler
                6. Ödeme koşulları ve son ödeme tarihi
                7. Faturanın içeriğinin kısa bir özeti

                Fatura İçeriği:
                {text}
                """,

                'email': """
                Bu belge "email" (e-posta) olarak sınıflandırılmıştır.
                Lütfen aşağıdaki e-postayı analiz et ve şu bilgileri çıkar:

                1. E-postanın kimden geldiği ve kime gönderildiği
                2. E-postanın tarihi ve saati
                3. E-postanın konusu
                4. E-postada belirtilen önemli bilgiler veya talepler
                5. E-postada bahsedilen kişiler, kurumlar veya projeler
                6. E-postanın tonu ve amacı
                7. E-postanın içeriğinin kısa bir özeti

                E-posta İçeriği:
                {text}
                """
            }

            # Sınıfa özel şablon varsa kullan, yoksa genel şablonu kullan
            prompt_template = class_specific_templates.get(document_class, """
            Bu belge "{document_class}" olarak sınıflandırılmıştır.
            Lütfen aşağıdaki belgeyi analiz et ve şu bilgileri çıkar:

            1. Belge tipi ve amacı nedir?
            2. Ana konusu veya içeriği nedir?
            3. Önemli anahtar-değer çiftleri (varsa)
            4. Belgedeki önemli varlıklar (kişiler, şirketler, tarihler vb.)
            5. Belgenin yapısı ve formatı nasıldır?
            6. Belgenin hedef kitlesi kimdir?
            7. Belgenin içeriğinin özeti

            Belge İçeriği:
            {text}
            """)

        # Analiz çalışanı
        def analyze_worker():
            try:
                # Analiz başlangıç zamanı
                start_time = time.time()
                print(f"LLM analizi başlıyor, metin uzunluğu: {len(text)} karakter")

                # Metni kısalt (çok uzunsa)
                max_chars = 500   # Maksimum 4000 karakter
                truncated_text = text[:max_chars] if len(text) > max_chars else text

                # Prompt oluştur
                prompt = prompt_template.format(document_class=document_class, text=truncated_text)

                # LLM yanıtı al
                if self.pipe is None:
                    result_queue.put({
                        'analysis': "LLM modeli yüklenemediği için analiz yapılamadı.",
                        'model': "None",
                        'error': "Model yüklenemedi"
                    })
                    return

                response = self.pipe(
                    prompt,
                    max_new_tokens=800,
                    temperature=0.1,
                    top_p=0.9,
                    do_sample=True
                )

                generated_text = response[0]['generated_text']

                # Prompt'u çıkar ve sadece yanıtı al
                response_text = generated_text[len(prompt):].strip()

                # İşlem süresini hesapla
                elapsed_time = time.time() - start_time
                print(f"LLM analizi tamamlandı: {len(response_text)} karakter, {elapsed_time:.2f} saniye")

                # Sonucu kuyruğa ekle
                result_queue.put({
                    'analysis': response_text,
                    'model': self.model,
                    'processing_time': elapsed_time
                })

            except Exception as e:
                print(f"LLM analiz hatası: {e}")
                exception_queue.put(e)

        # İş parçacığını başlat
        thread = threading.Thread(target=analyze_worker)
        thread.daemon = True
        thread.start()

        # Zaman aşımı ile bekle
        thread.join(timeout=self.timeout)

        # Thread hala çalışıyorsa zaman aşımına uğradı
        if thread.is_alive():
            print(f"LLM analizi zaman aşımına uğradı ({self.timeout} saniye)")
            return {
                'analysis': f"[LLM analizi zaman aşımına uğradı ({self.timeout} saniye). Belge çok karmaşık veya LLM işlem kapasitesini aşıyor olabilir.]",
                'model': self.model,
                'error': 'timeout'
            }

        # İstisna olup olmadığını kontrol et
        if not exception_queue.empty():
            e = exception_queue.get()
            return {
                'analysis': f"[LLM analiz hatası: {str(e)}]",
                'model': self.model,
                'error': str(e)
            }

        # Sonuçları döndür
        if not result_queue.empty():
            return result_queue.get()
        else:
            return {
                'analysis': "[LLM analizi tamamlandı ancak sonuç bulunamadı]",
                'model': self.model,
                'error': 'no_result'
            }

In [ ]:
class DocumentProcessingPipeline:
    def __init__(self, data_dir=None, model_dir=None):
        self.data_dir = data_dir
        self.model_dir = model_dir or os.path.join(os.getcwd(), 'models')

        # Çıktı dizinlerini oluştur
        os.makedirs(self.model_dir, exist_ok=True)

        print("Belge işleme pipeline'ı başlatılıyor...")

        # Sınıflandırıcıyı oluştur
        print("Swin Transformer modelini yüklüyor...")
        self.classifier = DocumentClassifier()

        # Eğitilmiş modeli yükle (varsa)
        best_model_path = os.path.join(os.getcwd(),'/content/drive/MyDrive/RVL_CDIP_small/best_document_classifier.pth')
        if os.path.exists(best_model_path):
            self.classifier.load_best_model(best_model_path)
        else:
            print("Hazır model bulunamadı. Yeni model eğitimi gerekecek.")

        print("Metin çıkarıcıyı başlatıyor...")
        self.extractor = DocumentTextExtractor()

        print("Belge analizörünü başlatıyor...")
        self.analyzer = DocumentAnalyzer()

        print("Pipeline hazır!")

    def train_classifier(self, train_dir=None, val_dir=None, batch_size=16, num_epochs=10, learning_rate=2e-5, progress_callback=None):
        print(f"Sınıflandırıcı eğitimi başlıyor...")

        # Eğitim ve doğrulama dizinlerini kontrol et
        train_dir = train_dir or os.path.join(self.data_dir, 'train')
        val_dir = val_dir or os.path.join(self.data_dir, 'val')

        if not os.path.exists(train_dir) or not os.path.exists(val_dir):
            print("Eğitim ve doğrulama dizinleri bulunamadı. Veri seti hazırlanıyor...")

            # Veri seti yapısını kontrol et
            has_subfolders = any(os.path.isdir(os.path.join(self.data_dir, d)) and d in CLASSES for d in os.listdir(self.data_dir))

            if has_subfolders:
                # Veri setini hazırla
                train_dir, val_dir, _ = prepare_dataset(self.data_dir)
            else:
                print("UYARI: Veri seti formatı doğru değil, model eğitimi başarısız olabilir!")
                train_dir = val_dir = self.data_dir

        # Veri artırma dönüşümlerini al
        train_transform = self.classifier.processor.get_train_transform()

        # Veri setlerini oluştur
        train_dataset = DocumentDataset(train_dir, self.classifier.processor, transform=train_transform, is_training=True)
        val_dataset = DocumentDataset(val_dir, self.classifier.processor)

        # Veri yükleyicileri oluştur
        num_workers = min(os.cpu_count() or 1, 4)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

        print(f"Eğitim veri seti: {len(train_dataset)} belge")
        print(f"Doğrulama veri seti: {len(val_dataset)} belge")
        print(f"Batch boyutu: {batch_size}, Epoch sayısı: {num_epochs}, İşçi sayısı: {num_workers}")

        # Belleği temizle
        torch.cuda.empty_cache()

        # Modeli eğit
        results = self.classifier.train(
            train_loader,
            val_loader,
            num_epochs=num_epochs,
            learning_rate=learning_rate,
            progress=progress_callback
        )

        # Sonuçları kaydet
        results_file = os.path.join(self.model_dir, 'training_results.json')
        with open(results_file, 'w') as f:
            json.dump({
                'train_losses': [float(loss) for loss in results['train_losses']],
                'val_accuracies': [float(acc) for acc in results['val_accuracies']],
                'val_f1_scores': [float(f1) for f1 in results['val_f1_scores']],
                'best_accuracy': float(results['best_accuracy']),
                'best_f1': float(results['best_f1']),
                'training_time_seconds': float(results['total_time'])
            }, f, indent=2)

        print(f"Eğitim sonuçları kaydedildi: {results_file}")

        return results

    def evaluate_classifier(self, test_dir=None, batch_size=16, progress_callback=None):
        print("Sınıflandırıcı değerlendirmesi başlıyor...")

        test_dir = test_dir or os.path.join(self.data_dir, 'test')

        # Test dizinini kontrol et
        if not os.path.exists(test_dir) and os.path.exists(os.path.join(self.data_dir, 'train')):
            print("Test dizini bulunamadı. Veri seti yeniden hazırlanıyor...")
            _, _, test_dir = prepare_dataset(self.data_dir)
        elif not os.path.exists(test_dir):
            # Test dizini yoksa, ana veri seti dizinini kullan
            test_dir = self.data_dir
            print(f"Test dizini bulunamadı, ana veri seti dizini kullanılıyor: {test_dir}")

        # Test veri setini oluştur
        test_dataset = DocumentDataset(test_dir, self.classifier.processor)

        # Test veri yükleyicisini oluştur
        num_workers = min(os.cpu_count() or 1, 4)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

        print(f"Test veri seti: {len(test_dataset)} belge")

        # Modeli değerlendir
        eval_results = self.classifier.evaluate(test_loader, progress=progress_callback)

        # Raporu kaydet
        report_df = pd.DataFrame(eval_results['report']).transpose()
        report_path = os.path.join(self.model_dir, 'classification_report.csv')
        report_df.to_csv(report_path)

        print(f"Değerlendirme raporu kaydedildi: {report_path}")

        return eval_results

    def process_document(self, document_path, skip_analysis=False):
        """
        Belge işleme pipeline'ını çalıştır:
        1. Görsel sınıflandırma
        2. Metin çıkarma
        3. İçerik analizi (gerekirse)
        """
        print(f"Belge işleniyor: {document_path}")

        results = {}

        # Adım 1: Görsel sınıflandırma
        print("Adım 1: Görsel sınıflandırma...")
        try:
            classification_result = self.classifier.predict(document_path)
            results['classification'] = classification_result

            doc_class = classification_result['class']
            confidence = classification_result['confidence']
            print(f"Belge sınıfı: {doc_class} (güven: {confidence:.4f})")
        except Exception as e:
            print(f"Sınıflandırma hatası: {e}")
            results['classification'] = {
                'class': 'error',
                'confidence': 0.0,
                'error': str(e)
            }
            doc_class = 'error'
            confidence = 0.0

        # Adım 2: Metin çıkarma (zaman aşımı koruması ile)
        print("Adım 2: Metin çıkarma...")
        try:
            # Zaman aşımı için bir süre belirle
            extraction_timeout = 30  # 30 saniye

            # Thread kullanarak zaman aşımını kontrol et
            def extract_with_timeout():
                return self.extractor.extract_text(document_path)

            import threading
            extraction_thread = threading.Thread(target=extract_with_timeout)
            extraction_thread.daemon = True

            extraction_result = None
            extraction_thread.start()
            extraction_thread.join(extraction_timeout)

            if extraction_thread.is_alive():
                print(f"Metin çıkarma zaman aşımına uğradı ({extraction_timeout} saniye)")
                # Zaman aşımına uğrarsa, varsayılan boş sonuç döndür
                extraction_result = {
                    'text': "[Metin çıkarma zaman aşımına uğradı]",
                    'metadata': {'error': 'timeout'}
                }
            else:
                extraction_result = extract_with_timeout()

            results['extraction'] = {
                'text': extraction_result['text'],
                'metadata': extraction_result['metadata']
            }

            text_length = len(extraction_result['text'])
            print(f"Çıkarılan metin uzunluğu: {text_length} karakter")

        except Exception as e:
            print(f"Metin çıkarma hatası: {e}")
            results['extraction'] = {
                'text': "[Metin çıkarma başarısız oldu]",
                'metadata': {'error': str(e)}
            }
            text_length = 0

        # Adım 3: İçerik analizi (opsiyonel)
        if not skip_analysis and confidence > 0.5 and text_length > 50:
            print("Adım 3: İçerik analizi...")
            try:
                analysis_result = self.analyzer.analyze_document(
                    results['extraction']['text'],
                    doc_class
                )
                results['analysis'] = analysis_result
                print("İçerik analizi tamamlandı.")
            except Exception as e:
                print(f"İçerik analizi hatası: {e}")
                results['analysis'] = {
                    'analysis': "[Analiz hatası oluştu]",
                    'error': str(e)
                }
        else:
            if skip_analysis:
                print("İçerik analizi atlandı (kullanıcı tercihiyle).")
            else:
                print("İçerik analizi atlandı (düşük güven veya yetersiz metin).")
            results['analysis'] = None

        print("Belge işleme tamamlandı.")

        return results

    def process_batch(self, folder_path, max_documents=None, output_csv=None, skip_analysis=False, progress_callback=None):
        """
        Bir klasördeki belgeleri toplu olarak işle
        """
        print(f"Toplu işleme başlatılıyor: {folder_path}")

        # Tüm uygun belgeleri bul
        documents = []
        for root, _, files in os.walk(folder_path):
            for file in files:
                if file.endswith(('.tif', '.jpg', '.png', '.jpeg', '.pdf')):
                    documents.append(os.path.join(root, file))

        # Maksimum belge sayısını kontrol et
        if max_documents and len(documents) > max_documents:
            documents = documents[:max_documents]

        print(f"İşlenecek belge sayısı: {len(documents)}")

        # Her belgeyi işle
        results = []

        for i, doc_path in enumerate(tqdm(documents, desc="Belgeler işleniyor")):
            if progress_callback:
                progress_callback((i / len(documents)) * 100, f"Belge işleniyor: {i+1}/{len(documents)}")

            try:
                result = self.process_document(doc_path, skip_analysis=skip_analysis)

                # Sonuçları kaydet
                results.append({
                    'path': doc_path,
                    'class': result['classification']['class'],
                    'confidence': result['classification']['confidence'],
                    'text_length': len(result['extraction']['text']),
                    'analysis': result['analysis']['analysis'] if result['analysis'] else None
                })

            except Exception as e:
                print(f"Hata: {e}")
                results.append({
                    'path': doc_path,
                    'error': str(e)
                })

        # Sonuçları CSV olarak kaydet
        if output_csv:
            df = pd.DataFrame(results)
            df.to_csv(output_csv, index=False)
            print(f"Sonuçlar kaydedildi: {output_csv}")

        return results

In [ ]:
class UnstructuredTextExtractor:
    def __init__(self):
        print("Unstructured metin çıkarıcı başlatılıyor...")
        # Zaman aşımı ekleyebiliriz
        self.timeout = 60  # 60 saniye

    def extract_text(self, document_path):
        """
        Unstructured kütüphanesi ile metin çıkarma işlemi
        Güvenlik için zaman aşımı ve hata yönetimi eklenmiştir
        """
        import threading
        import time
        from queue import Queue

        result_queue = Queue()
        exception_queue = Queue()

        def extraction_worker():
            try:
                # Hata ayıklama için başlangıç zamanı
                start_time = time.time()
                print(f"Metin çıkarma başlıyor: {document_path}")

                # Belgenin uzantısını kontrol et
                ext = os.path.splitext(document_path)[1].lower()

                # TIF dosyalarına özel işlem
                if ext in ['.tif', '.tiff']:
                    try:
                        # TIF dosyasını PIL ile açıp geçici bir JPEG olarak kaydet
                        from PIL import Image
                        print(f"TIF dosyası işleniyor: {document_path}")
                        img = Image.open(document_path)

                        # Geçici jpeg dosyası oluştur
                        temp_jpg = document_path + "_temp.jpg"
                        img.convert('RGB').save(temp_jpg)
                        print(f"Geçici JPEG oluşturuldu: {temp_jpg}")

                        # Geçici JPEG dosyasını işle
                        from unstructured.partition.auto import partition
                        from unstructured.staging.base import elements_to_text

                        print("Unstructured ile ayrıştırma başlıyor...")
                        elements = partition(temp_jpg)
                        text = elements_to_text(elements)

                        # Geçici dosyayı temizle
                        try:
                            os.remove(temp_jpg)
                        except:
                            pass

                    except Exception as e:
                        print(f"TIF işleme hatası: {e}")
                        text = f"[TIF işleme hatası: {str(e)}]"
                        elements = []
                else:
                    # Diğer dosya türleri için doğrudan partition kullan
                    try:
                        from unstructured.partition.auto import partition
                        from unstructured.staging.base import elements_to_text

                        print("Unstructured ile ayrıştırma başlıyor...")
                        elements = partition(document_path)
                        text = elements_to_text(elements)
                    except Exception as e:
                        print(f"Ayrıştırma hatası: {e}")
                        text = f"[Ayrıştırma hatası: {str(e)}]"
                        elements = []

                # İşlem süresini kontrol et
                elapsed_time = time.time() - start_time
                print(f"Metin çıkarma tamamlandı: {len(text)} karakter, {elapsed_time:.2f} saniye")

                # Sonuçları kuyruğa ekle
                result_queue.put({
                    'text': text,
                    'metadata': {
                        'num_elements': len(elements),
                        'processing_time': elapsed_time,
                        'file_path': document_path
                    }
                })

            except Exception as e:
                print(f"Metin çıkarma thread hatası: {e}")
                exception_queue.put(e)

        # İş parçacığını başlat
        thread = threading.Thread(target=extraction_worker)
        thread.daemon = True
        thread.start()

        # Zaman aşımı ile bekle
        thread.join(timeout=self.timeout)

        # Thread hala çalışıyorsa zaman aşımına uğradı
        if thread.is_alive():
            print(f"Metin çıkarma zaman aşımına uğradı ({self.timeout} saniye)")
            return {
                'text': f"[Metin çıkarma zaman aşımına uğradı ({self.timeout} saniye)]",
                'metadata': {'error': 'timeout', 'file_path': document_path}
            }

        # İstisna olup olmadığını kontrol et
        if not exception_queue.empty():
            e = exception_queue.get()
            return {
                'text': f"[Metin çıkarma hatası: {str(e)}]",
                'metadata': {'error': str(e), 'file_path': document_path}
            }

        # Sonuçları döndür
        if not result_queue.empty():
            return result_queue.get()
        else:
            return {
                'text': "[Metin çıkarma işlemi tamamlandı ancak sonuç bulunamadı]",
                'metadata': {'error': 'no_result', 'file_path': document_path}
            }


In [ ]:


class DocumentProcessingUI:
    def __init__(self, data_dir=None):
        # Varsayılan model dosyası
        self.model_path = os.path.join(os.getcwd(), '/content/drive/MyDrive/RVL_CDIP_small/best_document_classifier.pth')
        self.data_dir = data_dir

        # Sınıflandırıcıyı başlat
        self.classifier = DocumentClassifier()

        # OCR işleyicisini başlat
        self.extractor = UnstructuredTextExtractor()

        # LLM analiz modülünü başlat
        self.analyzer = DocumentAnalyzer()

        # Eğitilmiş model var mı kontrol et
        self.model_exists = os.path.exists(self.model_path)

        if self.model_exists:
            success = self.classifier.load_best_model(self.model_path)
            print(f"Model yükleme {'başarılı' if success else 'başarısız'}: {self.model_path}")
        else:
            print(f"UYARI: Eğitilmiş model bulunamadı: {self.model_path}")

        # Arayüzü oluştur
        self.app = self._create_ui()

    def _create_ui(self):
        with gr.Blocks(title="Belge İşleme Sistemi", theme=gr.themes.Soft()) as app:
            gr.Markdown("# 📄 Belge İşleme Sistemi")
            gr.Markdown("Swin Transformer ile belge sınıflandırma, OCR ve LLM analizi.")

            # Model durumunu göster
            if self.model_exists:
                gr.Markdown(f"✅ Eğitilmiş model otomatik olarak yüklendi: {self.model_path}")
            else:
                gr.Markdown(f"⚠️ Eğitilmiş model bulunamadı. Lütfen önce modeli eğitin.")

            with gr.Tab("Belge İşleme"):
                with gr.Row():
                    with gr.Column(scale=1):
                        # Belge yükleme bölümü
                        file_input = gr.File(label="Belge Seçin (PDF, TIF, JPG, PNG)")
                        with gr.Row():
                            classify_btn = gr.Button("Sadece Sınıflandır", variant="secondary")
                            process_btn = gr.Button("Sınıflandır & OCR", variant="secondary")
                            analyze_btn = gr.Button("Tam Analiz (Sınıflandırma+OCR+LLM)", variant="primary")

                        image_preview = gr.Image(type="filepath", label="Belge Önizleme")

                    with gr.Column(scale=1):
                        # Sonuç gösterimi bölümü
                        result_box = gr.Textbox(label="Sonuç Özeti", interactive=False, lines=2)
                        doc_class = gr.Textbox(label="Belge Sınıfı", interactive=False)
                        confidence = gr.Number(label="Doğruluk Oranı (%)", interactive=False)

                        # Sınıf olasılıkları için tablo
                        gr.Markdown("### Sınıf Olasılıkları")
                        top_classes = gr.Dataframe(
                            headers=["Sınıf", "Olasılık (%)"],
                            datatype=["str", "number"],
                            row_count=(5, "fixed"),
                            col_count=(2, "fixed"),
                            interactive=False
                        )

                        # OCR Sonuçları için metin kutusu
                        with gr.Accordion("Çıkarılan Metin (OCR)", open=False):
                            extracted_text = gr.Textbox(
                                label="Belge Metni",
                                interactive=False,
                                lines=10,
                                placeholder="Metin çıkarma sonuçları burada görünecek."
                            )
                            ocr_info = gr.Textbox(
                                label="OCR Bilgisi",
                                interactive=False,
                                lines=1
                            )

                        # LLM Analiz Sonuçları
                        with gr.Accordion("Belge Analizi (LLM)", open=False):
                            analysis_text = gr.Textbox(
                                label="LLM Analizi",
                                interactive=False,
                                lines=15,
                                placeholder="LLM analiz sonuçları burada görünecek."
                            )
                            analysis_info = gr.Textbox(
                                label="Analiz Bilgisi",
                                interactive=False,
                                lines=1
                            )

            # Sadece sınıflandırma fonksiyonu
            def classify_document(file):
                if file is None:
                    return None, "Lütfen bir belge yükleyin.", None, None, [], "", "", "", ""

                try:
                    # Sınıflandırma yap
                    result = self.classifier.predict(file.name)
                    doc_class_result = result['class']
                    confidence_score = result['confidence'] * 100
                    summary = f"Belge '{doc_class_result}' olarak {confidence_score:.2f}% doğrulukla sınıflandırıldı."

                    # Top 5 sınıf ve olasılıkları
                    top_5_classes = []
                    for cls, prob in result['sorted_probs'][:5]:
                        top_5_classes.append([cls, round(prob * 100, 2)])

                    return file.name, summary, doc_class_result, confidence_score, top_5_classes, "", "", "", ""
                except Exception as e:
                    error_msg = f"Sınıflandırma hatası: {str(e)}"
                    return None, error_msg, "Hata", 0, [], "", "", "", ""

            # Sınıflandırma ve OCR fonksiyonu
            def process_document(file):
                if file is None:
                    return None, "Lütfen bir belge yükleyin.", None, None, [], "", "", "", ""

                try:
                    # Önce sınıflandırma yap
                    classification_result = self.classifier.predict(file.name)
                    doc_class_result = classification_result['class']
                    confidence_score = classification_result['confidence'] * 100

                    # Top 5 sınıf ve olasılıkları
                    top_5_classes = []
                    for cls, prob in classification_result['sorted_probs'][:5]:
                        top_5_classes.append([cls, round(prob * 100, 2)])

                    # Sonra metin çıkarma yap
                    extraction_start = time.time()
                    extraction_result = self.extractor.extract_text(file.name)
                    extracted = extraction_result['text']
                    extraction_time = time.time() - extraction_start

                    # OCR bilgisi
                    ocr_info_text = f"OCR işlemi {extraction_time:.2f} saniyede tamamlandı. Metin uzunluğu: {len(extracted)} karakter."

                    # Özet oluştur
                    if "error" in extraction_result['metadata']:
                        summary = f"Belge '{doc_class_result}' olarak sınıflandırıldı. OCR HATASI: {extraction_result['metadata']['error']}"
                    else:
                        summary = f"Belge '{doc_class_result}' olarak {confidence_score:.2f}% doğrulukla sınıflandırıldı. OCR başarılı."

                    return file.name, summary, doc_class_result, confidence_score, top_5_classes, extracted, ocr_info_text, "", ""
                except Exception as e:
                    error_msg = f"İşleme hatası: {str(e)}"
                    return None, error_msg, "Hata", 0, [], "İşleme sırasında hata oluştu.", "", "", ""

            # Tam analiz fonksiyonu (Sınıflandırma + OCR + LLM Analizi)
            def analyze_document(file):
                if file is None:
                    return None, "Lütfen bir belge yükleyin.", None, None, [], "", "", "", ""

                try:
                    # 1. Sınıflandırma
                    classification_start = time.time()
                    classification_result = self.classifier.predict(file.name)
                    doc_class_result = classification_result['class']
                    confidence_score = classification_result['confidence'] * 100
                    classification_time = time.time() - classification_start

                    # Top 5 sınıf ve olasılıkları
                    top_5_classes = []
                    for cls, prob in classification_result['sorted_probs'][:5]:
                        top_5_classes.append([cls, round(prob * 100, 2)])

                    # 2. OCR ile metin çıkarma
                    extraction_start = time.time()
                    extraction_result = self.extractor.extract_text(file.name)
                    extracted_text = extraction_result['text']
                    extraction_time = time.time() - extraction_start

                    # OCR bilgisi
                    ocr_info_text = f"OCR işlemi {extraction_time:.2f} saniyede tamamlandı. Metin uzunluğu: {len(extracted_text)} karakter."

                    # 3. LLM ile analiz (yeterli metin varsa)
                    analysis_text = ""
                    analysis_info_text = ""

                    if len(extracted_text) > 50:
                        analysis_start = time.time()
                        analysis_result = self.analyzer.analyze_document(
                            extracted_text,
                            doc_class_result
                        )
                        analysis_text = analysis_result['analysis']
                        analysis_time = time.time() - analysis_start
                        analysis_info_text = f"LLM analizi {analysis_time:.2f} saniyede tamamlandı. Analiz uzunluğu: {len(analysis_text)} karakter."
                    else:
                        analysis_text = "Metin analizi için yeterli içerik bulunamadı (minimum 50 karakter gerekli)."
                        analysis_info_text = "Analiz yapılamadı: Yetersiz metin."

                    # Özet oluştur
                    summary = f"Belge '{doc_class_result}' olarak {confidence_score:.2f}% doğrulukla sınıflandırıldı."
                    if "error" in extraction_result['metadata']:
                        summary += f" OCR HATASI: {extraction_result['metadata']['error']}"
                    else:
                        summary += f" OCR ve LLM analizi tamamlandı."

                    return file.name, summary, doc_class_result, confidence_score, top_5_classes, extracted_text, ocr_info_text, analysis_text, analysis_info_text
                except Exception as e:
                    error_msg = f"İşleme hatası: {str(e)}"
                    return None, error_msg, "Hata", 0, [], "İşleme sırasında hata oluştu.", "", "Analiz yapılamadı.", ""

            # Olay bağlama: Dosya seçildiğinde önizleme güncellemesi
            file_input.change(
                lambda file: file.name if file else None,
                inputs=[file_input],
                outputs=[image_preview]
            )

            # Sınıflandırma butonu tıklandığında sadece sınıflandır
            classify_btn.click(
                classify_document,
                inputs=[file_input],
                outputs=[image_preview, result_box, doc_class, confidence, top_classes,
                         extracted_text, ocr_info, analysis_text, analysis_info]
            )

            # İşleme butonu tıklandığında sınıflandırma ve OCR yap
            process_btn.click(
                process_document,
                inputs=[file_input],
                outputs=[image_preview, result_box, doc_class, confidence, top_classes,
                         extracted_text, ocr_info, analysis_text, analysis_info]
            )

            # Analiz butonu tıklandığında tam analiz yap
            analyze_btn.click(
                analyze_document,
                inputs=[file_input],
                outputs=[image_preview, result_box, doc_class, confidence, top_classes,
                         extracted_text, ocr_info, analysis_text, analysis_info]
            )

        return app

    def launch(self, share=False):
        self.app.launch(share=share, debug=True)  # Debug modunu açtık

In [ ]:
# Google Colab için kullanım örneği
def start_colab_demo():
    # Google Drive'ı bağla
    mount_drive()

    # Veri seti yolu
    data_dir = '/content/drive/MyDrive/RVL_CDIP_small'

    # UI başlat - otomatik eğitim/yükleme yapılacak
    ui = DocumentProcessingUI(data_dir=data_dir)
    ui.launch(share=True)

# Ana program
if __name__ == "__main__":
    # Çalıştırma ortamını kontrol et
    try:
        import google.colab
        is_colab = True
    except:
        is_colab = False

    if is_colab:
        # Google Colab üzerinde çalışıyorsa
        start_colab_demo()
    else:
        # Normal bir Python ortamında çalışıyorsa
        data_dir = "RVL_CDIP_small"  # Yerel dizin yolu
        ui = DocumentProcessingUI(data_dir=data_dir)
        ui.launch()